In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import os
import itertools
import statistics

In [2]:
from ipynb.fs.full.particle_file_search import search_particle_filter
from ipynb.fs.full.behavior_segmentation_search import search_behavior_segmentation
from ipynb.fs.full.plot_animals import plot_function

In [1]:
import sys
sys.path.append("/Users/pierre.le.merre/dmclab-kostas/Developer/projects/dmcdb_py")

In [2]:
from readSessionsServer import SessionIterator

In [3]:
state_order = {
    'initRight': 0,
    'initLeft': 0,
    'mainRunLeft': 1,
    'turnMainToLeft': 2,
    'leftRun': 3,
    'leftReward': 4
}

In [ ]:
n_bins_per_seg = 10

#This loops through all the sessions (a.k.a. realizations).
#To just get a single session, write
# `sess = SessionIterator("Miniscope recordings").findSession()`
#instead of the loop.
for sess in SessionIterator("ipsilateral").findSessions():
    #Get the filtered speed and angular speed from the particle filter
    filtered = sess.getProcessedTable("Particle Filter")

    #Get the behavior segmentation per frame
    segmented = sess.getProcessedTable("Behavior segmentation", "per_frame")

    #For each frame, add an int 0-4 to match which phase of the trial the
    # frames falls in, or NaN if it's some other phase not in the list.
    state_order_series = pd.Series(state_order, name="state_order")
    segmented_relevant = segmented.join(state_order_series, on="state_name")

    #The x-coordinate is the state order (0 to 4), plus the progression through 
    #each frame (0 to 1) according to the behavior segmentation
    x_pos = segmented_relevant.spatial_progress + segmented_relevant.state_order

    #Only look at the speeds on frames which are not NaN, i.e. which fall into
    #one of the phases in `state_order`
    speeds = filtered.speed[np.isfinite(x_pos)]

    #Filter out NaNs and bin the x-coordinates
    bin_x = np.floor(x_pos[np.isfinite(x_pos)]*n_bins_per_seg).astype(int)

    #Per bin of x-coordiate...
    gby = (60*speeds).groupby([bin_x, segmented_relevant.segment_no]).mean().groupby(level=0)
   
    #...calulate the median
    medians = gby.median()

    #...and the the first and third quartile
    low_q = gby.quantile(0.25)
    high_q = gby.quantile(0.75)

    #The number of trials is approximated by the number of rewards at the left port
    N_trials = len(segmented[segmented.state_name == "leftReward"].segment_no.unique())
    
    #Plot these values
    plt.figure(figsize=(7, 3))
    plt.plot(medians)
    plt.fill_between(medians.index, low_q, high_q, alpha=.2)
    plt.title(f"{sess} (N={N_trials} trials)")
    labels = ["Turn at initiation", "Run in main arm", "Turn to left", "Run in left arm", "At reward"]
    for i in range(1, 5):
        plt.axvline(i*n_bins_per_seg, color='k', linestyle='--')
    plt.xticks((np.arange(5)+0.5)*n_bins_per_seg, labels, rotation=15, ha="right")
    plt.ylabel("Average speed (cm/s)")
    plt.ylim(-5, 30)
    sns.despine()
    plt.show()